In [ ]:
!pip install pandas numpy matplotlib seaborn statsmodels pandas-gbq catboost scikit-learn
import pandas as pd
import os
import pandas_gbq as pd_gbq
import seaborn as sns
import numpy as np
import statsmodels
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn
import catboost as cb
from sklearn.model_selection import train_test_split

from digital_model import train_country_newbies_model
from elast_simulation import simulate_budget_elasticity

In [ ]:
import pandas_gbq as pd_gbq

with open("digital_data_set_querry.SQL", "r", encoding="utf-8") as f:
    sql = f.read()

df = pd_gbq.read_gbq(sql)

In [3]:
result = train_country_newbies_model(
    df,
    seed=42,
    holdout_days=30,
    half_life_days=60,
    top_n_countries=15,
    target_column = 'vertical_newbies_delivery_paid',
    catboost_params={"iterations": 3000, "learning_rate": 0.05}
)

In [4]:
result['metrics_overall']

,rmse,mae,wmape_pct,rows,unique_days,unique_countries,holdout_start,last_date,train_last_date
0,44.337465,24.446948,5.642645,450,30,15,2025-09-02,2025-10-01,2025-09-01


In [5]:
result['feature_importance']

,feature,importance
0,cpi_raw,16.156641
1,country_name,12.683159
2,eff_raw,10.310621
3,cost_total,8.972722
4,cost_modelled_courier_lag14,7.348977
5,cost_modelled_rh_passenger,4.750837
6,cost_modelled_rh_passenger_lag1,3.697731
7,cost_modelled_courier_roll7,3.669778
8,cost_modelled_rh_passenger_roll14,3.378181
9,cost_modelled_courier_lag1,2.892243


In [6]:
elast_paid = simulate_budget_elasticity(
    model=result["model"],
    df_model=result["df_model"],
    feature_columns=result["feature_columns"],
    categorical_features=result["categorical_features"],
    target_col="vertical_newbies_delivery_paid",
    start_date=result["holdout_start"],
    end_date=result["last_date"],
    budg_down=0.90,
    budg_up=1.10,
    efficiency_mode="safe",   # эффективность «заморожена» прокси из базы
)

In [7]:
elast_paid.sort_values(by='elasticity_up',ascending = False)

,country_name,actual,base_pred,avg_cost,pred_budg_down,delta_down,elasticity_down,pred_budg_up,delta_up,elasticity_up
10,Nigeria,209.166667,190.308563,3745.763916,184.659042,-5.649521,0.296861,213.650009,23.341446,1.226505
7,Indonesia,344.733333,334.517395,7929.779297,328.095978,-6.421417,0.191961,370.848572,36.331177,1.086077
5,Egypt,415.600000,396.937897,6525.430176,362.105164,-34.832733,0.877536,439.644135,42.706238,1.075892
1,Brazil,506.733333,489.419861,15569.826172,445.115662,-44.304199,0.905239,540.269287,50.849426,1.038973
13,South Africa,66.600000,53.338223,4883.801270,45.998886,-7.339336,1.375999,58.665936,5.327713,0.998855
14,Казахстан,645.033333,636.083069,5123.773438,582.793579,-53.289490,0.837776,697.806519,61.723450,0.970368
0,Bolivia,98.866667,84.050606,4341.099609,76.293541,-7.757065,0.922904,92.191414,8.140808,0.968560
4,Ecuador,185.366667,170.132797,3764.539062,153.461639,-16.671158,0.979891,185.529190,15.396393,0.904963
3,Colombia,832.500000,794.847412,11342.303711,733.771912,-61.075500,0.768393,865.956360,71.108948,0.894624
11,Pakistan,1163.000000,1127.113525,7547.296387,1016.656372,-110.457153,0.980000,1213.276489,86.162964,0.764457


In [8]:
df['newbies_total'] = df['vertical_newbies_delivery_paid']+df['vertical_newbies_delivery_organic']

In [9]:
result_total = train_country_newbies_model(
    df,
    seed=42,
    holdout_days=30,
    half_life_days=60,
    top_n_countries=15,
    target_column = 'newbies_total',
    catboost_params={"iterations": 3000, "learning_rate": 0.05}
)

In [10]:
result_total['metrics_overall']

,rmse,mae,wmape_pct,rows,unique_days,unique_countries,holdout_start,last_date,train_last_date
0,64.288646,35.735495,4.70884,450,30,15,2025-09-02,2025-10-01,2025-09-01


In [11]:
elast_total = simulate_budget_elasticity(
    model=result_total["model"],
    df_model=result_total["df_model"],
    feature_columns=result_total["feature_columns"],
    categorical_features=result_total["categorical_features"],
    target_col="newbies_total",
    start_date=result_total["holdout_start"],
    end_date=result_total["last_date"],
    budg_down=0.90,
    budg_up=1.10,
    efficiency_mode="safe",   # эффективность «заморожена» прокси из базы
)

In [12]:
elast_total.sort_values(by='elasticity_up',ascending = False)

,country_name,actual,base_pred,avg_cost,pred_budg_down,delta_down,elasticity_down,pred_budg_up,delta_up,elasticity_up
7,Indonesia,824.266667,792.723572,7929.779297,774.432434,-18.291138,0.230738,891.596741,98.873169,1.247259
0,Bolivia,147.633333,131.616898,4341.099609,118.472282,-13.144615,0.998703,147.558029,15.941132,1.211177
13,South Africa,99.866667,83.911583,4883.801270,71.664940,-12.246643,1.459470,93.927025,10.015442,1.193571
1,Brazil,890.566667,868.189514,15569.826172,795.428833,-72.760681,0.838074,961.435852,93.246338,1.074032
10,Nigeria,454.233333,404.796173,3745.763916,385.216583,-19.579590,0.483690,447.645905,42.849731,1.058551
14,Казахстан,1145.666667,1122.426392,5123.773438,1028.338379,-94.088013,0.838256,1237.537476,115.111084,1.025556
3,Colombia,1359.433333,1324.361816,11342.303711,1208.121460,-116.240356,0.877708,1455.826904,131.465088,0.992667
4,Ecuador,315.033333,298.301056,3764.539062,268.954285,-29.346771,0.983797,326.509613,28.208557,0.945641
5,Egypt,703.966667,680.454163,6525.430176,620.643005,-59.811157,0.878989,741.902588,61.448425,0.903050
2,Chile,151.466667,132.857330,2936.004395,120.760910,-12.096420,0.910482,143.490631,10.633301,0.800355


In [13]:
elast_total.to_csv('elast_for_total_newbies.csv')

In [ ]:
holdout_start = result['holdout_start']
last_date = result['last_date']
train_last_date = ['2025-08-31']
df_model = result['df_model'].copy()
n_train_days = df_model.loc[df_model['day'] < holdout_start, 'day'].nunique()
n_train_rows = X_train.shape[0] if 'X_train' in globals() else len(df_model.loc[df_model['day'] < holdout_start])
print(f"Train unique days: {n_train_days}")
print(f"Train rows: {n_train_rows}")
print(f"holdout_start: {holdout_start}, last_date: {last_date}, train_last_date: {train_last_date}")